# Verbless clauses and היה-clauses in the BHSA

In this script information is collected about all the all the clauses with a subject and predicate complement, with and without the verb היה in the [BHSA](https://etcbc.github.io/bhsa/).
The data are saved in a csv file.

In [1]:
import sys, os, csv, collections
import pandas as pd

In [2]:
from tf.app import use
A = use('bhsa', hoist=globals())

A.displaySetup(extraFeatures='kind')
A.displaySetup(extraFeatures='prs')

	connecting to online GitHub repo annotation/app-bhsa ... connected
Using TF-app in C:\Users\geitb/text-fabric-data/annotation/app-bhsa/code:
	rv1.2=#5fdf1778d51d938bfe80b37b415e36618e50190c (latest release)
	connecting to online GitHub repo etcbc/bhsa ... connected
Using data in C:\Users\geitb/text-fabric-data/etcbc/bhsa/tf/c:
	rv1.6=#bac4a9f5a2bbdede96ba6caea45e762fe88f88c5 (latest release)
	connecting to online GitHub repo etcbc/phono ... connected
Using data in C:\Users\geitb/text-fabric-data/etcbc/phono/tf/c:
	r1.2 (latest release)
	connecting to online GitHub repo etcbc/parallels ... connected
Using data in C:\Users\geitb/text-fabric-data/etcbc/parallels/tf/c:
	r1.2 (latest release)
   |     0.00s No structure info in otext, the structure part of the T-API cannot be used


A dictionary is made in which for each book its genre is specified.

In [3]:
genre_dict = {}

prose = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy', 'Joshua', 'Judges', '1_Samuel', '2_Samuel', '1_Kings', '2_Kings', 'Jonah', 'Ruth', 'Esther', 'Daniel', 'Ezra', 'Nehemiah', '1_Chronicles', '2_Chronicles']
prophecy = ['Isaiah', 'Jeremiah', 'Ezekiel', 'Hosea', 'Joel', 'Obadiah', 'Micah', 'Zephaniah', 'Haggai', 'Zechariah', 'Malachi', 'Amos', 'Nahum', 'Habakkuk']
poetry = ['Song_of_songs','Proverbs','Ecclesiastes', 'Lamentations', 'Psalms', 'Job']

for genre in [prose, prophecy, poetry]:
    for book in genre:
        if book in prose:
            genre_dict[book] = 'prose'
        elif book in prophecy:
            genre_dict[book] = 'prophecy'
        elif book in poetry:
            genre_dict[book] = 'poetry'

A dictionary is made in which EBH and LBH (and 'other') is specified for each book.

In [4]:
ebh_lbh_dict = {}

ebh = ['Genesis', 'Exodus', 'Leviticus', 'Numbers', 'Deuteronomy', 'Joshua', 'Judges', '1_Samuel', '2_Samuel', '1_Kings', '2_Kings']
lbh = ['Esther', 'Daniel', 'Ezra', 'Nehemiah', '1_Chronicles', '2_Chronicles']
for book in ebh:
    ebh_lbh_dict[book] = 'ebh'
for book in lbh:
    ebh_lbh_dict[book] = 'lbh'

double_books = {'1_Samuel': 'Samuel', '2_Samuel' : 'Samuel', '2_Kings' : 'Kings', '1_Kings': 'Kings', '1_Chronicles' : 'Chronicles', '2_Chronicles' : 'Chronicles'}

In a few helper functions is is calculated whether a clause is a main clause, a subordinate clause (3 different varieties) or undecided. For more information, check the file main_suboordinate_clauses.ipynb in the folder "Various".

In [5]:
def in_dep_calc(cl):  
      
    in_dep = ''        
    if F.rela.v(cl) == 'ReSu': # is the clause resumptive?
        moth_obj = E.mother.f(cl)[0]
        in_dep = rela_calc(moth_obj)
    else:
        in_dep = rela_calc(cl) # does the clause have a dependent CCR?

    if in_dep == '':
    	words = L.d(cl, 'word') # is there a wayyiqtol?
    	for word in words:
            if F.vt.v(word) == 'wayq':
                in_dep += 'Main'
                        
    if in_dep == '':  # if everything else does not give a result, we look at the CARC
        cl_atoms = L.d(cl, 'clause_atom')
        in_dep = carc_calc(cl_atoms)
        
    return(in_dep)


def carc_calc(cl_atoms):
    in_dep_c = ''
    carc = F.code.v(cl_atoms[0])
    if 999 > int(carc) > 499:
        in_dep_c += 'SubAdv'
    elif int(carc) in {0, 999}:
        in_dep_c = 'Main'
    elif 17 > int(carc) > 9:
        in_dep_c += 'SubAdv'
    elif 75 > int(carc) > 50:
        in_dep_c += 'SubAdv'
    elif 168 > int(carc) > 99:
        in_dep_c += 'Main'
    elif 500 > int(carc) > 299:
        in_dep_c += 'Main'
    elif int(carc) in {200, 201}:         
        while F.code.v(cl_atoms[0]) in {200, 201}:
            cl_atoms = E.mother.f(cl_atoms[0])
        carc = F.code.v(cl_atoms[0])
        if 999 > int(carc) > 499:
            in_dep_c += 'SubAdv'
        elif int(carc) in {0, 999}:
            in_dep_c = 'Main'
        elif 17 > int(carc) > 9:
            in_dep_c += 'SubAdv'
        elif 75 > int(carc) > 50:
            in_dep_c += 'SubAdv'
        elif 168 > int(carc) > 99:
            in_dep_c += 'Main'
        elif 500 > int(carc) > 299:
            in_dep_c += 'Main'
        elif int(carc) in {220, 221, 222, 223}:
            in_dep_c += 'Undc'
        
    else:
        in_dep_c += 'Undc'
        
    return(in_dep_c)


def rela_calc(cl):
    in_dep_r = ''
    ccr = F.rela.v(cl)
    if ccr in {'Subj', 'Objc', 'Cmpl', 'PreC', 'Voct', 'Frnt'}:
        in_dep_r += 'SubArg'
    elif ccr in {'Attr', 'RgRc', 'Spec'}:
        in_dep_r += 'SubMod'
    elif ccr in {'Adju', 'PrAd'}:
        in_dep_r += 'SubAdv'
    elif ccr == 'Coor':
        moth_obj = E.mother.f(cl)[0]
        if F.otype.v(moth_obj) in {'word', 'phrase'}:
            in_dep_r += 'SubMod'
        else:
            while F.rela.v(moth_obj) == 'Coor':
                moth_obj = E.mother.f(moth_obj)[0]
            ccr = F.rela.v(cl)
            if ccr in {'Subj', 'Objc', 'Cmpl', 'PreC', 'Voct', 'Frnt'}:
                in_dep_r += 'SubArg'
            elif ccr in {'Attr', 'RgRc', 'Spec'}:
                in_dep_r += 'SubMod'
            elif ccr in {'Adju', 'PrAd'}:
                in_dep_r += 'SubAdv'
                
        if in_dep_r == '':
            if F.otype.v(moth_obj) != 'clause':
                in_dep_r += 'SubMod'
            else:
                cl_atoms = L.d(moth_obj, 'clause_atom')
                in_dep_r = carc_calc(cl_atoms)
                
    return(in_dep_r)

The function mother_clause_calc retrieves the tense of the mother of that clause(atom).

In [6]:
def mother_tense_calc(cl):
    
    mother_tense = ''
    cl_atoms = L.d(cl, 'clause_atom')
    moth = E.mother.f(cl_atoms[0])
    if len(moth) == 0:
        mother_tense += 'no_mother'
    else: 
        if F.otype.v(moth[0]) in {'word', 'phrase'}:
            mother_tense += F.otype.v(moth[0])
        else:
            cl = L.u(moth[0], 'clause')[0]
            if F.kind.v(cl) == 'NC':
                mother_tense += 'nominal'
            elif F.kind.v(cl) == 'WP':
                mother_tense += 'no_pred'
                        
            else:
                phrases = L.d(cl, 'phrase')
                pred = False
                prec = False
                for phr in phrases:
                    if F.function.v(phr) in {'Pred', 'PreS', 'PreO'}:
                        pred = True
                        pred_phr = phr
                    elif F.function.v(phr) in {'PreC', 'PtcO'}:
                        prec = True
                        prec_phr = phr
                if pred == True:
                    words = L.d(pred_phr, 'word')
                    for word in words:
                        if F.sp.v(word) == 'verb':
                            mother_tense += F.vt.v(word)
                elif prec == True:
                    words = L.d(prec_phr, 'word')
                    for word in words:
                        if F.sp.v(word) == 'verb':
                            mother_tense += F.vt.v(word)
                            
    return(mother_tense)

A list with all the different phrase functions is created. This is used in the main program.

In [7]:
extra_cl_funcs = set()

for phrase in F.otype.s('phrase'):
    phr_func = F.function.v(phrase)
    if not phr_func in {'Subj', 'Pred', 'PreC'}:
        extra_cl_funcs.add(phr_func)
                    
extra_cl_funcs = sorted(list(extra_cl_funcs))                
extra_cl_funcs.append('Unkn')

In [8]:
print(extra_cl_funcs)

['Adju', 'Cmpl', 'Conj', 'EPPr', 'ExsS', 'Exst', 'Frnt', 'IntS', 'Intj', 'Loca', 'ModS', 'Modi', 'NCoS', 'NCop', 'Nega', 'Objc', 'PrAd', 'PrcS', 'PreO', 'PreS', 'PtcO', 'Ques', 'Rela', 'Supp', 'Time', 'Voct', 'Unkn']


Of each clause it it determined whether it is a copula clause and its information is stored in the dict cop_clauses.

In [9]:
cl_dict = {}

n_par = 0

# loop over all the clauses
for cl in F.otype.s('clause'):
    feat_list = []
    
    # select clauses of kind "NC" (nominal clause) and "VC" (verbal clause)
    cl_kind = F.kind.v(cl)
    if cl_kind in {'NC', 'VC'}: 
        
        # make list with phrase nodes of the clause
        phrases = L.d(cl, 'phrase')
        phr_funcs = [F.function.v(phr) for phr in phrases]
        
        # collect clauses with subject and predicate complement 
        if 'Subj' in phr_funcs and 'PreC' in phr_funcs:
            cl_type = ''
            
            #define the clause types "hyh" and "nom"
            if cl_kind == 'VC' and 'Pred' in phr_funcs:
                cl_type += 'hyh'
            elif cl_kind == 'NC':
                cl_type += 'nom'
                
            if cl_type in {'hyh', 'nom'}:
                
                # collect basic info about clause
                feat_list.append(cl_type) # type of clause: hyh or nom
                feat_list.append(str(cl)) # clause id
                bo, ch, ve = T.sectionFromNode(cl)
                feat_list.append(bo) # book
                feat_list.append(str(ch)) # chapter
                feat_list.append(str(ve)) # verse
                feat_list.append(F.typ.v(cl)) # clause type of database
                feat_list.append(F.rela.v(cl) + '_') # clause relation
                
                # eg if book is '1_Samuel', add 'Samuel' to feat_list
                if bo in double_books: 
                    feat_list.append(double_books[bo])
                else:
                    feat_list.append(bo)
                
                # ebh_lbh
                if bo in ebh_lbh_dict.keys():
                    feat_list.append(ebh_lbh_dict[bo])
                else:
                    feat_list.append('other')
                    
                feat_list.append(genre_dict[bo]) #genre
                
                feat_list.append(F.txt.v(cl)[-1]) # Q, D, N
            
                if phr_funcs.index('Subj') < phr_funcs.index('PreC'): #subj and prec order
                    feat_list.append('SP')
                else:
                    feat_list.append('PS')
                    
                # which conjunction
                phr_types = [F.typ.v(phr) for phr in phrases]
                if 'CP' in phr_types: 
                    conj = phrases[phr_types.index('CP')]
                    words = L.d(conj, 'word')
                    words_lex = '_'.join([F.lex.v(wo) for wo in words])
                    feat_list.append(words_lex)
                else:
                    feat_list.append('no_conj')
                
                # clause length, in verbal clauses the Pred is not counted
                if cl_kind == 'VC':
                    feat_list.append(str(len(phrases) - 1)) 
                else:
                    feat_list.append(str(len(phrases)))
                
                subj_pos = phr_funcs.index('Subj')
                feat_list.append(F.typ.v(phrases[subj_pos])) #phrase type of subject
                feat_list.append(F.det.v(phrases[subj_pos])) #determination of subject
                
                # length of subj in words
                words_subj = L.d(phrases[subj_pos], 'word') # length of subject in words
                subj_len = len(words_subj)
                for wo in words_subj:                    
                    if not F.prs.v(wo) in {'absent', 'n/a'}:
                        subj_len += 1
                feat_list.append(str(subj_len))
                
                prec_pos = phr_funcs.index('PreC')
                feat_list.append(F.typ.v(phrases[prec_pos])) #phrase type of prec
                feat_list.append(F.det.v(phrases[prec_pos])) #determination of prec
                
                # length of prec in words
                words_prec = L.d(phrases[prec_pos], 'word')
                prec_len = len(words_prec)
                for wo in words_prec:                    
                    if not F.prs.v(wo) in {'absent', 'n/a'}:
                        prec_len += 1
                feat_list.append(str(prec_len))
            
                # main or subordinate clause
                feat_list.append(in_dep_calc(cl))
                                     
                # negation in clause
                phr_types = [F.typ.v(phr) for phr in phrases]
                if 'NegP' in phr_types:
                    feat_list.append('neg')
                else:
                    feat_list.append('non_neg')
                
                # aramaic or hebrew
                words = L.d(cl, 'word')
                feat_list.append(F.language.v(words[0]))
                
                # collect info (function) about other phrases in clause (present (1) or absent (0))
                for item in extra_cl_funcs:   
                    if item in phr_funcs:
                        feat_list.append('1')
                    else:
                        feat_list.append('0')
                
                # tense ot the mother of the clause_atom
                feat_list.append(mother_tense_calc(cl))
                    
                cl_dict[cl] = feat_list

The data are saved in a csv [file](hyh_nom_bib.csv).

In [10]:
df_hjh_nom = pd.DataFrame(cl_dict).T

header = ['cl_type', 'cl_id', 'book', 'chapter', 'verse', 'clause_type_etcbc', 'clause_rela', 'book2', 'ebh_lbh', 'genre', 'txt_type', 's_p_order', 'CP', 'cl_len', 'subj_type', 'subj_det', 'subj_len', 'pc_type', 'pc_det', 'pc_len', 'main_sub', 'nega', 'language']

for item in extra_cl_funcs:
    header.append(item)
header.append('mother')

df_hjh_nom.columns = header 

df_hjh_nom.to_csv("hyh_nom_bib.csv", index = False)

print(df_hjh_nom.head)

<bound method NDFrame.head of        cl_type   cl_id          book chapter verse clause_type_etcbc  \
427554     hyh  427554       Genesis       1     2              WXQt   
427555     nom  427555       Genesis       1     2              NmCl   
427569     hyh  427569       Genesis       1     6              ZYqX   
427592     nom  427592       Genesis       1    11              NmCl   
427597     nom  427597       Genesis       1    12              NmCl   
...        ...     ...           ...     ...   ...               ...   
515599     nom  515599  2_Chronicles      36     5              NmCl   
515614     nom  515614  2_Chronicles      36     9              NmCl   
515622     nom  515622  2_Chronicles      36    11              NmCl   
515671     nom  515671  2_Chronicles      36    23              NmCl   
515672     nom  515672  2_Chronicles      36    23              NmCl   

       clause_rela       book2 ebh_lbh  genre  ... PreO PreS PtcO Ques Rela  \
427554         NA_     Gen